Load required packages 

In [2]:
import os 
import numpy as np 
import h5py
import scipy.io
import matlab_helpers as mh 
import tensorflow as tf 
from keras.utils import to_categorical

Read the names of all mat-files 

In [3]:
root_matdata_dir = 'D:/Dropbox/Python/MLmodels/Datasets/CarModPow/'
out_ml_dir = root_matdata_dir + 'ml_data_MLP/'
if not os.path.exists(out_ml_dir):
    os.makedirs(out_ml_dir)

clean_in_data_dir = root_matdata_dir + 'level65_dBspl_clean/'
valid_datadirs = ['Chut', 'HighWhistle', 'Rumble', 'Tchatter', 'Wheek', 'Whine']
calls2use = ['Chut', 'Rumble', 'Wheek', 'Whine']

all_files_nameonly= []
CarMod_data_list= []

for cur_call_dir in valid_datadirs:
    cur_call_path = clean_in_data_dir + cur_call_dir + '/' 
    # print(cur_call_path)
    cur_dir_files = [f for f in os.listdir(cur_call_path) if os.path.isfile(os.path.join(cur_call_path, f))]
    all_files_nameonly = all_files_nameonly + cur_dir_files
    CarMod_data_list = CarMod_data_list + [cur_call_path + f for f in cur_dir_files]
    

print(f"--> all_files_nameonly: len lines= {len(all_files_nameonly)}, first line = {all_files_nameonly[0]}")
print(f"--> CarMod_data_list: len lines= {len(CarMod_data_list)}, first line = {CarMod_data_list[0]}")

out_allfiles_txt_fname = out_ml_dir + 'all_CarPow_data_list.txt'
with open(out_allfiles_txt_fname, 'w') as f:
    for line in CarMod_data_list:
        f.write(f"{line}\n")

--> all_files_nameonly: len lines= 1605, first line = CarMod_psd_Chut_2_Feb_07_2022_51861688_ms_101198_101787.mat
--> CarMod_data_list: len lines= 1605, first line = D:/Dropbox/Python/MLmodels/Datasets/CarModPow/level65_dBspl_clean/Chut/CarMod_psd_Chut_2_Feb_07_2022_51861688_ms_101198_101787.mat


Read Mat files and then create .h5 files for training 

In [4]:
out_CarMod_data_file = out_ml_dir + 'all_CarMod_data.h5'
data_CarMod_x = []
pre_search_str = 'clean/'
post_search_str = '/CarMod_'

for fName in CarMod_data_list:
    data = mh.loadmat(fName)
    data_CarMod_x.append(np.ravel(data["CarMod_power"]["CarMod_power"]))
    
data_label_name = [item[item.rfind(pre_search_str)+len(pre_search_str):item.rfind(post_search_str)] for item in CarMod_data_list]
data_label_y = len(calls2use)*np.ones((len(data_CarMod_x),1))
unq_vals, unq_counts = np.unique(data_label_y, return_counts=True)
print(dict(zip(unq_vals,unq_counts)))

for ind, cur_call in zip(np.arange(len(data_label_name)),data_label_name):
    if cur_call in calls2use: 
        data_label_y[ind,0] = calls2use.index(cur_call)

data_CarMod_x = np.array(data_CarMod_x)
data_label_y = np.array(data_label_y).astype(int)

print(f"data_CarMod_x={type(data_CarMod_x)}&{len(data_CarMod_x)},data_label_y={type(data_label_y)}&{data_label_y.shape},")

if (not os.path.exists(out_CarMod_data_file)):
    print("Saving file" + out_CarMod_data_file)
    hf = h5py.File(out_CarMod_data_file, "w")
    hf.create_dataset('data_CarMod_x',data=data_CarMod_x)
    hf.create_dataset('data_label_y',data=data_label_y)
    hf.close()
else: 
    print("File (" + out_CarMod_data_file + ") already exists")




KeyboardInterrupt: 